In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torchvision import datasets, transforms
#from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes

In [25]:
#!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 9.9 MB/s eta 0:00:00


In [2]:
#training controls
batch_size = 20
epochs = 2
training_size = 0.7
learning_rate = 0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]

# input image dimensions
img_rows, img_cols = 268, 182

In [3]:
# the data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [4]:
#opening the dataset
dataset = csv.reader(open("MovieGenre.csv",encoding="utf8",errors='replace'), delimiter=",")

#skipping the header line
next(dataset)

['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [15]:
import zipfile as zf
files = zf.ZipFile("SampleMoviePosters.zip", 'r')
files.extractall()
files.close()

In [16]:
#the list of image files in SampleMoviePosters folder
flist=glob.glob('SampleMoviePosters/*.jpg')  

In [17]:
#extracting the data from the CSV file
for imdbId, Link, Title, Score, Genre, Poster in dataset:
    if(Score!=""):
        if(len((int(imdbId),float(Score)))==2):
            tempY.append((int(imdbId),float(Score)))

In [18]:
#setting the length of training data
length=int(len(flist)*training_size)
length

697

In [27]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('\\')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
            else:
                x_test.append(img)
                y_test.append(z[1])
    i+=1

In [28]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [29]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [30]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (715, 3, 268, 182)
715 train samples
303 test samples


In [33]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

TypeError: expected np.ndarray (got Tensor)